# Title: Keras_Embedding_Residual_Gate_Model
## Description: 
다수의 범주형 변수를 `임베딩` 레이어로 변환하고, Dropout-Gate-Residual 연결 구조를 통해 과적합을 방지하며 깊은 신경망을 학습하는 아키텍처.
 - `why` : 컴퓨터가 이해할 수 없는 문자나 범주형 데이터(Category)를 `one-hot encoding` 하지 않고 의미를 가진 밀집된 벡터(Dense Vector)로 변환
## Input: 
- `Keras Input Layer` 
- `Embedding Dimensions Info`
## Output: 
- `Keras Model Output Tensor`
## Check Point: 
- TensorFlow 2.x 이상 필요, GPU 가속 권장

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def create_embedding_residual_block(input_layer, input_dims, start_neurons=32, num_residual_blocks=5):
    """
    input_layer: Keras Input Tensor
    input_dims: 리스트 형태. 각 컬럼(Feature)별 고유값의 개수(vocab size).
                수치형 변수는 해당 위치에 None 혹은 0으로 표기하여 임베딩 제외.
    start_neurons: 임베딩 차원 및 Dense 레이어의 기본 노드 수
    """
    embeddings = []
    
    # 1. Input Processing (Embedding vs Dense)
    for i, vocab_size in enumerate(input_dims):
        if vocab_size and vocab_size > 0:
            # 범주형: (Batch, ) -> (Batch, Embedding_Dim)
            slice_layer = input_layer[:, i] # i번째 컬럼 선택
            x = layers.Embedding(input_dim=vocab_size, output_dim=start_neurons)(slice_layer)
        else:
            # 수치형: (Batch, 1) -> (Batch, Neurons)
            # i:i+1을 쓴 이유는 2차원 형태를 유지하기 위해서 / Dense 레이어는 입력이 최소 2차원이어야 작동하기 쉽다.
            slice_layer = input_layer[:, i:i+1]
            x = layers.Dense(start_neurons)(slice_layer)
            
        embeddings.append(x)
    
    # 모든 피처 벡터 연결
    combined_layer = layers.concatenate(embeddings)
    
    # 2. Residual Blocks with Gating
    x = combined_layer
    
    for _ in range(num_residual_blocks):
        # Dropout
        x_dropout = layers.Dropout(0.2)(x)
        # Gating Mechanism
        # 정보의 흐름을 제어하는 문지기 역할
        gate = layers.Dense(x.shape[-1], activation='sigmoid')(x_dropout)
        gate_output = layers.Multiply()([x, gate]) # element-wise multiplication
        
        # Concatenate & Dense processing
        x_concat = layers.concatenate([x, gate_output])
        
        # Residual Connection
        # Vanishing Gradient 방지 (add original input to output)
        residual = layers.Dense(x.shape[-1], activation='relu')(x_concat)
        x = layers.Add()([x, residual])

    # --- 3. Output Headers ---
    # 안정성을 위해 여러 output head의 평균을 사용하는 앙상블 효과 (선택사항)
    outputs = []
    for _ in range(5):
        out = layers.Dense(1)(x) # 회귀(Regression) 기준
        outputs.append(out)
        
    # 출력값 평균 (Averaging)
    final_output = layers.Average()(outputs)
    
    return final_output

# --- 테스트 코드 ---
# 범주형(100개), 범주형(50개), 수치형(0)
dummy_input_dims = [100, 50, 0] 
input_tensor = Input(shape=(3,)) # 컬럼이 3개인 데이터 가정
output_tensor = create_embedding_residual_block(input_tensor, dummy_input_dims)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None)            │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None)            │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 1)         │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 32)        │      3,200 │ get_item_4[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 32)        │      1,600 │ get_item_5[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │         64 │ get_item_6[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 96)        │          0 │ embedding_2[0][0… │
│ (Concatenate)       │                   │            │ embedding_3[0][0… │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 96)        │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 96)        │      9,312 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 96)        │          0 │ concatenate_1[0]… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 192)       │          0 │ concatenate_1[0]… │
│ (Concatenate)       │                   │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 96)        │     18,528 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 96)        │          0 │ concatenate_1[0]… │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 96)        │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 96)        │      9,312 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 96)        │          0 │ add[0][0],        │
│ (Multiply)          │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 192)       │          0 │ add[0][0],        │
│ (Concatenate)       │                   │            │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 144,549 (564.64 KB)

 Trainable params: 144,549 (564.64 KB)

 Non-trainable params: 0 (0.00 B)

## How to Use
- `input_dims` 리스트 설정이 핵심입니다. 데이터프레임의 각 컬럼 순서대로 `df[col].nunique()` 값을 리스트로 만들어 전달해야 합니다. 수치형 변수 구간에는 `0`을 넣으세요.
- 회귀 문제가 아닌 분류 문제인 경우, 마지막 `final_output` 직전의 `Dense` 레이어 활성화 함수를 `softmax`나 `sigmoid`로 변경해야 합니다.

## Troubleshooting
- **Shape Error**: `input_dims`의 길이와 `input_layer`의 shape[1] (컬럼 수)가 정확히 일치해야 합니다. 불일치 시 `IndexError`나 `Shape mismatch` 오류가 발생합니다.
- **Slow Training**: 임베딩 차원(`start_neurons`)이 너무 크거나 잔차 블록 수(`num_residual_blocks`)가 많으면 파라미터 수가 급증합니다. 리소스에 맞춰 조절하세요.